In [1]:
import fitz
path = "/Users/poth/Documents/Fall2022/DEAN690/TCDS1/2A8.pdf"
pdf = fitz.open(path)
pdf.metadata

{'format': 'PDF 1.5',
 'title': '2A8_Revision_22',
 'author': 'DOT/FAA',
 'subject': 'Revision 22',
 'keywords': 'Lavia',
 'creator': 'Microsoft® Word 2016',
 'producer': 'Microsoft® Word 2016',
 'creationDate': "D:20210714152310-07'00'",
 'modDate': "D:20210714152310-07'00'",
 'encryption': None}

In [2]:
#pc = pdf.page_count
#print(pc)

In [3]:
# This function is to get all the [x0, y0, x1, y1, word, block_no, line_no, word_no] from the given TCDS file.

In [4]:
def get_all_text(data):
    pc = data.pageCount
    word_box = []
    for i in range(0,pc):
        ct = data.loadPage(i)
        gw = ct.getText('words')
        word_box += gw
    
    return word_box

In [5]:
all_text = get_all_text(pdf)
print(all_text)

[(72.02400207519531, 719.7630004882812, 89.56499481201172, 731.7689819335938, 'Page', 0, 0, 0), (91.79698944091797, 719.7630004882812, 105.08099365234375, 731.7689819335938, 'No.', 0, 0, 1), (124.33999633789062, 719.7630004882812, 128.83999633789062, 731.7689819335938, '1', 0, 1, 0), (146.89999389648438, 719.7630004882812, 151.39999389648438, 731.7689819335938, '2', 0, 2, 0), (169.33999633789062, 719.7630004882812, 173.83999633789062, 731.7689819335938, '3', 0, 3, 0), (191.89999389648438, 719.7630004882812, 196.39999389648438, 731.7689819335938, '4', 0, 4, 0), (214.3699951171875, 719.7630004882812, 218.8699951171875, 731.7689819335938, '5', 0, 5, 0), (236.92999267578125, 719.7630004882812, 241.42999267578125, 731.7689819335938, '6', 0, 6, 0), (259.3699951171875, 719.7630004882812, 263.8699951171875, 731.7689819335938, '7', 0, 7, 0), (281.92999267578125, 719.7630004882812, 286.42999267578125, 731.7689819335938, '8', 0, 8, 0), (72.02400207519531, 733.2030029296875, 88.67399597167969, 745

In [6]:
import pandas as pd
output = pd.DataFrame(all_text, columns=["x0", "y0", "x1", "y1", "word", "block_no", "line_no", "word_no"])
output.to_excel('output.xlsx')

In [7]:
def word_line(in_put):
    index = 0
    word_list = []
    output = [[]]
    for i in range(0,len(in_put)-1):
        if(in_put[i][-1]+1 == in_put[i+1][-1]):
            word_list += [in_put[i][-4]]  
        else:
            if(in_put[i+1][-1]==0):
                word_list += [in_put[i][-4]]
                word_list = [' '.join(word_list)]
                word_list += [in_put[i][-3],in_put[i][-2],in_put[i][-1]]
                word_list.insert( 0, in_put[i-in_put[i][-1]][0])
                word_list.insert( 1, in_put[i-in_put[i][-1]][1])
                word_list.insert( 2, in_put[i][2])
                word_list.insert( 3, in_put[i][3])
            print(word_list)

            output[index] += word_list
            index += 1
            word_list = []
            output += [[]]
    
    return output

In [8]:
out = word_line(all_text)

[72.02400207519531, 719.7630004882812, 105.08099365234375, 731.7689819335938, 'Page No.', 0, 0, 1]
[124.33999633789062, 719.7630004882812, 128.83999633789062, 731.7689819335938, '1', 0, 1, 0]
[146.89999389648438, 719.7630004882812, 151.39999389648438, 731.7689819335938, '2', 0, 2, 0]
[169.33999633789062, 719.7630004882812, 173.83999633789062, 731.7689819335938, '3', 0, 3, 0]
[191.89999389648438, 719.7630004882812, 196.39999389648438, 731.7689819335938, '4', 0, 4, 0]
[214.3699951171875, 719.7630004882812, 218.8699951171875, 731.7689819335938, '5', 0, 5, 0]
[236.92999267578125, 719.7630004882812, 241.42999267578125, 731.7689819335938, '6', 0, 6, 0]
[259.3699951171875, 719.7630004882812, 263.8699951171875, 731.7689819335938, '7', 0, 7, 0]
[281.92999267578125, 719.7630004882812, 286.42999267578125, 731.7689819335938, '8', 0, 8, 0]
[72.02400207519531, 733.2030029296875, 104.2439956665039, 745.208984375, 'Rev. No.', 1, 0, 1]
[122.05999755859375, 733.2030029296875, 131.1199951171875, 745.2089

In [9]:
type(out)

list

In [10]:
# Get the X1 of first given word

In [11]:
def get_word_x1(df, word):
    
    for i in range(0,len(df)):
        if(df[i][4] == word):
            return df[i][2]

In [12]:
fuel_x1 = get_word_x1(out, 'Fuel')
print(fuel_x1)

106.07099914550781


In [13]:
# define the line is [key_term, Model_header, information] to [1,2,3]

In [26]:
def detect_khi(in_put):
    index = 0
    temp_list= []
    out_put = [[]]
    for i in range(0,len(in_put)-1):
        if(in_put[i][0] <= fuel_x1) & (in_put[i][-1] <= 5 ):
            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
            temp_list.insert(8, 'key_term')
            out_put[index] = temp_list
            temp_list = []
            out_put += [[]]
            index += 1
        elif(in_put[i][0] <= fuel_x1) & (in_put[i][4].find('Model') > 0):
            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
            temp_list.insert(8, 'header')
            out_put[index] = temp_list
            temp_list = []
            out_put += [[]]
            index += 1
        else:
            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
            temp_list.insert(8, 'info')
            out_put[index] = temp_list
            temp_list = []
            out_put += [[]]
            index += 1
    print(out_put)
    return out_put

In [27]:
#def detect_khi(in_put):
#    index = 0
#    temp_list= []
#    out_put = [[]]
#    for i in range(0,len(in_put)-1):
#        if(in_put[i][0] <= fuel_x1)& (in_put[i][-1] <= 5 ):
#            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
#            temp_list.insert(8, 'key_term')
#            out_put[index] = temp_list
#            temp_list = []
#            out_put += [[]]
#            index += 1
#        elif(in_put[i][0] <= fuel_x1) & (in_put[i][4].find('Model') > 0):
#            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
#            temp_list.insert(8, 'header')
#            out_put[index] = temp_list
#            temp_list = []
#            out_put += [[]]
#            index += 1
#        else:
#            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
#            temp_list.insert(8, 'info')
#            out_put[index] = temp_list
#            temp_list = []
#            out_put += [[]]
#            index += 1
#    print(out_put)
#    return out_put

In [28]:
kt = detect_khi(out)
kt

[[72.02400207519531, 719.7630004882812, 105.08099365234375, 731.7689819335938, 'Page No.', 0, 0, 1, 'key_term'], [124.33999633789062, 719.7630004882812, 128.83999633789062, 731.7689819335938, '1', 0, 1, 0, 'info'], [146.89999389648438, 719.7630004882812, 151.39999389648438, 731.7689819335938, '2', 0, 2, 0, 'info'], [169.33999633789062, 719.7630004882812, 173.83999633789062, 731.7689819335938, '3', 0, 3, 0, 'info'], [191.89999389648438, 719.7630004882812, 196.39999389648438, 731.7689819335938, '4', 0, 4, 0, 'info'], [214.3699951171875, 719.7630004882812, 218.8699951171875, 731.7689819335938, '5', 0, 5, 0, 'info'], [236.92999267578125, 719.7630004882812, 241.42999267578125, 731.7689819335938, '6', 0, 6, 0, 'info'], [259.3699951171875, 719.7630004882812, 263.8699951171875, 731.7689819335938, '7', 0, 7, 0, 'info'], [281.92999267578125, 719.7630004882812, 286.42999267578125, 731.7689819335938, '8', 0, 8, 0, 'info'], [72.02400207519531, 733.2030029296875, 104.2439956665039, 745.208984375, 'R

[[72.02400207519531,
  719.7630004882812,
  105.08099365234375,
  731.7689819335938,
  'Page No.',
  0,
  0,
  1,
  'key_term'],
 [124.33999633789062,
  719.7630004882812,
  128.83999633789062,
  731.7689819335938,
  '1',
  0,
  1,
  0,
  'info'],
 [146.89999389648438,
  719.7630004882812,
  151.39999389648438,
  731.7689819335938,
  '2',
  0,
  2,
  0,
  'info'],
 [169.33999633789062,
  719.7630004882812,
  173.83999633789062,
  731.7689819335938,
  '3',
  0,
  3,
  0,
  'info'],
 [191.89999389648438,
  719.7630004882812,
  196.39999389648438,
  731.7689819335938,
  '4',
  0,
  4,
  0,
  'info'],
 [214.3699951171875,
  719.7630004882812,
  218.8699951171875,
  731.7689819335938,
  '5',
  0,
  5,
  0,
  'info'],
 [236.92999267578125,
  719.7630004882812,
  241.42999267578125,
  731.7689819335938,
  '6',
  0,
  6,
  0,
  'info'],
 [259.3699951171875,
  719.7630004882812,
  263.8699951171875,
  731.7689819335938,
  '7',
  0,
  7,
  0,
  'info'],
 [281.92999267578125,
  719.7630004882812,

In [29]:
for i in range(0, len(kt)):
    print(kt[i])

[72.02400207519531, 719.7630004882812, 105.08099365234375, 731.7689819335938, 'Page No.', 0, 0, 1, 'key_term']
[124.33999633789062, 719.7630004882812, 128.83999633789062, 731.7689819335938, '1', 0, 1, 0, 'info']
[146.89999389648438, 719.7630004882812, 151.39999389648438, 731.7689819335938, '2', 0, 2, 0, 'info']
[169.33999633789062, 719.7630004882812, 173.83999633789062, 731.7689819335938, '3', 0, 3, 0, 'info']
[191.89999389648438, 719.7630004882812, 196.39999389648438, 731.7689819335938, '4', 0, 4, 0, 'info']
[214.3699951171875, 719.7630004882812, 218.8699951171875, 731.7689819335938, '5', 0, 5, 0, 'info']
[236.92999267578125, 719.7630004882812, 241.42999267578125, 731.7689819335938, '6', 0, 6, 0, 'info']
[259.3699951171875, 719.7630004882812, 263.8699951171875, 731.7689819335938, '7', 0, 7, 0, 'info']
[281.92999267578125, 719.7630004882812, 286.42999267578125, 731.7689819335938, '8', 0, 8, 0, 'info']
[72.02400207519531, 733.2030029296875, 104.2439956665039, 745.208984375, 'Rev. No.', 

In [30]:
# sincet the detect_khi function can only define the first line of the headers.
# This function will mark all the other lines from 'info' to 'header'

In [31]:
def block_header(in_put):
    for i in range(0, len(in_put)-5):
        if(in_put[i][8] == 'header') & (in_put[i][5] == in_put[i+1][5]):
            in_put[i+1][8] = 'header'
            

In [32]:
bkt = block_header(kt)

In [33]:
# print all the words which have been marked as 'key_term'

In [34]:
d=[]
for i in range(0, len(kt)-1):
    if(kt[i][-1] == 'key_term'):
        d.append(kt[i][-5])
df=pd.DataFrame(d)
writer=pd.ExcelWriter('keyterm.xlsx',engine='xlsxwriter')
df.to_excel(writer, sheet_name='1', index=False)
writer.save()

In [35]:
Info=[]
for i in range(0, len(kt)-1):
    if(kt[i][-1] == 'key_term'):
        print(kt[i][-5])

Page No.
Rev. No.
and Federal Aviation Regulations.
Type Certificate Holder
Type Certificate Record of
Name Change
Engine
Fuel
Engine Limits
Propeller and Propeller Limits
Airspeed Limits (CAS)
Center Gravity Range
Empty Weight
Center of Gravity Range
Datum
Leveling Means
2A8
Maximum Weight
No. of Seats
Maximum Cargo
Fuel Capacity
Oil Capacity
Control Surface Movements
Manufacturer's Serial Numbers
Engine
Fuel
Engine Limits
Propeller and Propeller Limits
Airspeed Limits (CAS)
Center Gravity Range
Empty Weight
Datum
Leveling Means
Maximum Weight
Number of Seats
Maximum Cargo
Fuel Capacity
Oil Capacity
Control Surface Movements
Manufacturer's Serial No.
Engine
Fuel
Engine Limits
Propeller and Propeller Limits
Airspeed Limits (CAS)
Center Gravity Range
2A8
Empty Weight
Center of Gravity Range
Datum
Leveling Means
Maximum Weight
Number of Seats
Maximum Cargo
Fuel Capacity
Oil Capacity
Control Surface Movements
Manufacturer's Serial Numbers
DATA PERTINENT TO ALL MODELS
Certification Basis
P

In [24]:
# print all the words which have been marked as 'header'

In [36]:
for i in range(0, len(kt)-1):
    if(kt[i][-1] == 'header'):
        print(kt[i][-5])

I.- Model PA-25, 1 PCLM (Normal Category), Approved January 6, 1959.
II. - Model PA-25-235, 1 PCLM (Normal Category), Approved October 12, 1962.
III. - Model PA-25-260, 1 PCLM (Normal Category Only), Approved April 19, 1967.


In [23]:
# The coming function is tried to add page into the df. 
# There's some error of the function and the page no is maybe not necessary.

In [59]:
#def add_page(df):
#    p = 1
#    for i in range(0,len(df)-1):
#        if(df[i][5] <= df[i+1][5]):
#            df.insert(-1, p)
#        else:
#            p += 1
#            df.insert(-1, p)
#            print(df)

In [66]:
#add_page(out)